# Performance Predictive Model Demo

## Generate ONNX model using timm library
### You can generate your custom model from PyTorch or TensorFlow into ONNX

In [ ]:
from timm.models import create_model
import torch
from timm.models import create_model
from timm.data import resolve_data_config


def get_model(model_name, model_dtype, device, drop_rate):
    model = create_model(
        model_name, pretrained=False, in_chans=3, exportable=True, drop_rate=drop_rate, scriptable=False)
    model.to(device=device, dtype=model_dtype)
    data_config = resolve_data_config({}, model=model)
    input_size = data_config['input_size']
    num_classes = model.num_classes
    return model, input_size, num_classes

def get_input(batch_size, input_size, data_dtype, device):
    dummy_inputs = torch.randn(
        (batch_size,) + input_size, device=device, dtype=data_dtype)
    return dummy_inputs

def to_onnx(model, dummy_inputs, onnx_file):
    with torch.no_grad():
        model.eval()
        torch.onnx.export(model, dummy_inputs, onnx_file, verbose=True, export_params=False)
    torch.cuda.empty_cache()


## Give model name and batch size

In [ ]:
MODEL_NAME = "vgg16"
BATCH_SIZE = 128

In [ ]:
device = torch.device('cpu')
model_dtype = torch.float32
model, input_size, num_classes = get_model(MODEL_NAME, model_dtype, device, 0.0)
dummy_inputs = get_input(BATCH_SIZE, input_size, model_dtype, device)
onnx_file = f'{MODEL_NAME}_{BATCH_SIZE}.onnx'
to_onnx(model, dummy_inputs, onnx_file)


# TraPPM prediction on single A100 GPU 40GB HBM
#### code: github.com/karthickai/trappm

In [ ]:
import trappm

In [ ]:
trappm.predict("vgg16_128.onnx")